In [15]:
import pandas as pd
from sqlalchemy import create_engine

# Sample data
data = {
    'ConsumerID': [1, 2, 3],
    'Name': ['Sahil Rai', 'Rahul Yadav', 'Guddu Tyagi'],
    'Address': ['123 Mount St, Pune, Maharashtra, 411043', '456 Element St, Raipur, Rajasthan, 678901', '789 Darvin St, Panji, Goa, 456789'],
    'ContactNumber': ['985-675-5758', '982-562-8762', '953-369-3533'],
    'EmailAddress': ['Sahil@gmail.com', 'Rahul@gmail.com', 'Guddu@gmail.com'],
    'AccountNumber': ['AC501', 'BK406', 'CM89'],
    'MeterNumber': ['M001', 'M002', 'M003'],
    'TariffPlan': ['Plan1', 'Plan2', 'Plan3'],
    'ConsumptionHistory': ['History1', 'History2', 'History3'],
    'PaymentStatus': ['Paid', 'Pending', 'Paid']
}

# Create DataFrame
sample_data = pd.DataFrame(data)

# Define database connection strings
source_db_connection_string = 'sqlite:///source.db'
target_db_connection_string = 'sqlite:///target.db'

# Connect to source database
source_engine = create_engine(source_db_connection_string)

# Connect to target database
target_engine = create_engine(target_db_connection_string)

# Load sample data into source database
sample_data.to_sql('consumer_data_table', source_engine, if_exists='replace', index=False)

# Define SQL query to extract consumer data
query = """
    SELECT 
        ConsumerID,
        Name,
        Address,
        ContactNumber,
        EmailAddress,
        AccountNumber,
        MeterNumber,
        TariffPlan,
        ConsumptionHistory,
        PaymentStatus
    FROM 
        consumer_data_table
"""

# Extract data from source database
consumer_data = pd.read_sql(query, source_engine)

# Perform data transformation and mapping
consumer_data[['First Name', 'Last Name']] = consumer_data['Name'].str.split(' ', 1, expand=True)
#consumer_data[['Address Line 1', 'Address Line 2', 'City', 'State', 'Zip Code']] = 
Address1 = consumer_data['Address'].str.split(',', expand=True)
Address2 = Address1[0].str.split(' ', expand =True)
consumer_data["Address Line 1"] = Address2[0]
consumer_data["Address Line 2"] = Address2[1] +" " + Address2[2]
consumer_data["City"] = Address1[1]
consumer_data["State"] = Address1[2]
consumer_data["ZipCode"] = Address1[3]



consumer_data.rename(columns={'ContactNumber': 'Phone Number', 'EmailAddress': 'Email Address'}, inplace=True)

# Drop columns that are no longer needed
consumer_data.drop(['Name', 'Address', 'AccountNumber', 'MeterNumber', 'TariffPlan', 'ConsumptionHistory','PaymentStatus'], axis=1, inplace=True)
newData = consumer_data[['ConsumerID', 'First Name', 'Last Name', 'Address Line 1', 'Address Line 2', 'City', 'State', 'ZipCode', 'Phone Number', 'Email Address']]
# Load mapped data into target database
newData.to_sql('SMART360_Consumer', target_engine, if_exists='replace', index=False)

# Validate by reading from target database and displaying the data
validated_data = pd.read_sql('SELECT * FROM SMART360_Consumer', target_engine)
print(validated_data)


   ConsumerID First Name Last Name Address Line 1 Address Line 2     City  \
0           1      Sahil       Rai            123       Mount St     Pune   
1           2      Rahul     Yadav            456     Element St   Raipur   
2           3      Guddu     Tyagi            789      Darvin St    Panji   

          State  ZipCode  Phone Number    Email Address  
0   Maharashtra   411043  985-675-5758  Sahil@gmail.com  
1     Rajasthan   678901  982-562-8762  Rahul@gmail.com  
2           Goa   456789  953-369-3533  Guddu@gmail.com  
